<a href="https://colab.research.google.com/github/DrMiracle/Colab-Projects/blob/main/data%20analysis/different%20methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# завдання 1
import math
import numpy as np

def f(x): # phi(x)
    return np.sqrt(x+9) + 1

def iteration(x0, epsilon, function):
    x = function(x0)
    while(np.abs(x - x0) > epsilon):
        x0 = x
        x = function(x)
    return x

x_iter = iteration(1, 0.0001, f)
x_iter


4.7015725256021685

In [ ]:
# завдання 2
import numpy as np

A = [[36,-12,9],
    [-12,29,12],
    [9,12,27.5]]

b = [-162,24,-90.5]

def jacobi(A, b, epsilon):
    A, b = np.array(A, dtype=float), np.array(b, dtype=float)
    x_prev = np.zeros(b.shape, dtype=float)
    x_curr = b / A.diagonal()
    n_iter = 0
    while np.linalg.norm(x_prev - x_curr) > epsilon:
        x_prev = x_curr.copy()
        for i in range(b.shape[0]):
            x_curr[i] = x_prev[i] + (b[i] - np.multiply(A[i], x_prev).sum())/A[i, i]
        n_iter += 1
    return x_curr, n_iter
x, iterations = jacobi(A, b, 0.001)
print("x = {}, Кількість ітерацій: {}".format(x, iterations))

x = [-4.01281777 -0.01716183 -1.97047811], Кількість ітерацій: 19


In [ ]:
# завдання 3 (Лагранж)
import numpy as np

f = {}
f[-1]=3.141592654
f[-0.75]=2.418858406
f[-0.5]=2.094395102
f[-0.25]=1.823476582
f[0]=1.570796327
f[0.25]=1.318116072
f[0.5]=1.047197551
f[0.75]=0.7227342478
f[1]=0

def get_poly_values(poly_coefs, x):
    power_matrix = x.reshape((-1, 1))**np.arange(poly_coefs.shape[1]).reshape((1, -1))
    return power_matrix.dot(poly_coefs.reshape((-1, 1)))

def lagrange(f_dict):
    x = np.fromiter(f_dict.keys(), dtype=float)
    f_x = np.fromiter(f_dict.values(), dtype=float)
    res = np.zeros((1, x.shape[0]))
    for k in range(x.shape[0]):
        vec = None
        for j in range(x.shape[0]):
            if j == k:
                continue
            cur_vec = np.array([x[j] / (x[j] - x[k]), 1 / (x[k] - x[j])]).reshape((-1, 1))
            if vec is None:
                vec = cur_vec.reshape((1, -1))
            else:
                M = np.matmul(cur_vec, vec)
                vec = np.zeros((1, vec.shape[1] + 1))
                vec[0, :-1] += M[0]
                vec[0, 1:] += M[1]
        res += f_x[k] * vec
    return res

coefs = lagrange(f)
get_poly_values(coefs, np.array([-0.85, -0.35, 0.35, 0.85]))

array([[2.6156082 ],
       [1.92929835],
       [1.21229431],
       [0.52598445]])

In [ ]:
# завдання 3 (Чебишова)
import numpy as np

f = np.array([2.967059728, 2.617993878, 2.268928027, 1.919862177, 1.570796327, 1.221730477, 0.8726646262, 0.5235987752, 0.1745329253])[::-1]

x = np.array([-0.85, -0.35, 0.35, 0.85])

def get_poly_values(poly_coefs, x):
    power_matrix = x.reshape((-1, 1))**np.arange(poly_coefs.shape[1]).reshape((1, -1))
    return power_matrix.dot(poly_coefs.reshape((-1, 1)))

def get_coskj(N):
    j_coefs = np.array([np.pi * (2*j+1) / (2*N+2) for j in range(N+1)]).reshape((1, -1))
    kj = np.arange(N+1).reshape(-1, 1) * j_coefs
    return np.cos(kj)

def get_tk(N):
    T = np.zeros((N+1, N+1))
    T[0, 0] = 1
    T[1, 1] = 1
    for j in range(2, N+1):
        T[j, 1:j+1] = 2 * T[j-1, :j]
        T[j] -= T[j-2]
    return T

def cheb(f_values):
    f_values = np.array(f_values).reshape((-1, 1))
    N = f_values.shape[0] - 1
    cos_kj = get_coskj(N)
    a_k = np.matmul(cos_kj, f_values) # (n+1, 1)
    tk = get_tk(N) # (n+1, n+1)
    mm = np.matmul(tk.T, 2*a_k/(N+1))
    coefs = np.zeros(N+1)
    coefs[0] = a_k[0,0] / (N+1)
    coefs[1:] = mm[1:,0]
    return coefs

cheb(f)
get_poly_values(cheb(f).reshape((1, -1)), x)

array([[2.58421506],
       [1.92864467],
       [1.21294798],
       [0.55737759]])

In [ ]:
# завдання 4 метод прямокутників
import numpy as np

def f(x):
    return 1/(x+2)
def rectangles_method(function, a, b, n):
    h = (b - a)/n
    integral = 0
    for i in range(n):
        integral += function(a + h/2 + i*h)
    return integral * h

real_integral = np.log(1+2) - np.log(-1+2)
print("Метод прямокутників: {}".format(rectangles_method(f, -1, 1, 10)))
print("Реально можлива похибка: {}".format(np.abs(real_integral-rectangles_method(f, -1, 1, 10))))

Метод прямокутників: 1.0971420940086145
Реально можлива похибка: 0.001470194659495272


In [ ]:
# завдання 4 метод трапецій
import numpy as np

def f(x):
    return 1/(x+2)
def trapezoid_method(function, a, b, n):
    h = (b - a)/n
    integral = 0
    for i in range(n):
        integral += function(a + i*h) + function(a + (i+1)*h)
    return integral * h / 2

real_integral = np.log(1+2) - np.log(-1+2)
print("Метод трапецій: {}".format(trapezoid_method(f, -1, 1, 10)))
print("Реально можлива похибка: {}".format(np.abs(real_integral-trapezoid_method(f, -1, 1, 10))))

Метод трапецій: 1.1015623265623267
Реально можлива похибка: 0.0029500378942168837


In [ ]:
# завдання 4 метод Сімпсона
import numpy as np

def f(x):
    return 1/(x+2)
def simpson_method(function, a, b, n):
    h = (b - a)/n
    integral = 0
    for i in range(n):
        integral += function(a + i*h) + 4*function(a + i*h + h/2) + function(a + (i + 1)*h)
    return integral * h / 6

real_integral = np.log(1+2) - np.log(-1+2)
print("Метод Сімпсона: {}".format(simpson_method(f, -1, 1, 10)))
print("Реально можлива похибка: {}".format(np.abs(real_integral-simpson_method(f, -1, 1, 10))))

Метод Сімпсона: 1.098615504859852
Реально можлива похибка: 3.2161917422612873e-06
